In [1]:
import pandas as pd
import numpy as np
import keras
import jieba
import re

Using TensorFlow backend.


In [2]:
#读取数据
train_data = pd.read_csv('dp_utf8.csv', error_bad_lines=False,lineterminator='\n',skipinitialspace=True)
train_data

/Users/anthonymao08/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Review_ID,Merchant,Rating,Score_taste,Score_environment,Score_service,Price_per_person,Time,Num_thumbs_up,Num_ response,Content_review,Reviewer,Reviewer_value,Reviewer_rank,Favorite_foods
0,377313283,炳胜品味(珠江新城店),3,3,3,2,空,10月23日,2,1,来广州出差，住在附近的康莱德酒店，晚上开完会就慕名而来，大概七点四十分的样子到的，人还有很多...,胖子界的瘦子吴,5,1,\r
1,376404966,炳胜品味(珠江新城店),4,3,4,3,空,10月19日,2,0,?·??·??前言本来是朋友约我们来的，结果朋友临时有事走不开，就我俩在这儿吃了。?·??·...,快快到盆里来,6,1,\r
2,374370927,炳胜品味(珠江新城店),4,3,3,2,空,10月9日,0,0,朋友结婚选择在这里摆酒，在猎德A出口出来，直行转入兴盛路？再直行，走到天汇广场，过马路，再一...,Yyg喵,3,1,\r
3,377270662,炳胜品味(珠江新城店),4,3,3,3,155,10月23日,0,0,据说是个老子号，听名字就感觉很广东??，很正宗的样子！！这家分店打听了一下，说是地儿最大，环...,肥螳螂_,4,1,干炒牛河 炳胜荔枝鸡 \r
4,377184095,炳胜品味(珠江新城店),4,3,3,3,空,10月23日,0,0,周末不煮饭的日子，偶来啦??。中午12点多到店，3楼满了??让到2楼，2楼说要到1楼拿号??...,hhy_0919,4,1,\r
5,376654771,炳胜品味(珠江新城店),4,3,4,3,空,10月20日,1,0,环境特别优雅，一进门让人感觉特别舒服，价位适中。推荐几道菜#瑶柱佛手瓜煲龙#汤喝起来特别鲜甜...,噜啦啦璐璐,5,1,\r
6,371978056,炳胜品味(珠江新城店),5,4,4,3,空,9月28日,8,0,晚上看小蛮腰，坐公交坐到这块，然后搜了一下想说那晚饭就他家吧。大厅还算比较宽敞，晚上好像一直...,哆啦瞄瞄,7,1,\r
7,372562660,炳胜品味(珠江新城店),4,3,4,4,空,10月1日,2,0,交通便利，昨天是放假前一天，不敢开车去，所以选择了地铁，在APM剧院站上来步行10分钟就到了...,Henry莫,6,1,\r
8,373441960,炳胜品味(珠江新城店),4,3,3,3,空,10月5日,5,0,晚上差不多七点半到的那里，要排队，表示生意还真的挺好的，推测味道也应该挺好的。排了差不多半小...,趟七,5,1,脆皮叉烧拼秘制黑叉烧 豉油皇鹅肠 \r
9,375713033,炳胜品味(珠江新城店),3,1,4,2,90,10月15日,0,0,先说环境不错！不会想其他分店桌与桌之间靠得很近！这间位置很宽松，坐得也听舒服，不过灯光有点暗...,JMEAAKN,4,1,招牌菠萝包 脆皮叉烧拼秘制黑叉烧 古法烧鹅 \r


In [3]:
train_data=train_data.drop(['Merchant','Review_ID','Score_taste','Score_environment','Score_service','Price_per_person','Time','Reviewer','Reviewer_value','Reviewer_rank','Num_thumbs_up','Num_ response'],axis=1)

In [4]:
#data_column=list(train_data.columns)
train_data = train_data.drop('Favorite_foods\r',axis=1)

In [5]:
train_data['Rating'].describe()

count    467455.000000
mean          4.216292
std           0.962893
min           1.000000
25%           4.000000
50%           4.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

In [6]:
sample = train_data.sample(n=50000, replace=False, axis=0)
sample.describe()
dataset_1 = train_data[train_data['Rating'] == 1].sample(n=10000, replace=False, axis=0)
dataset_2 = train_data[train_data['Rating'] == 2].sample(n=10000, replace=False, axis=0)
dataset_3 = train_data[train_data['Rating'] == 3].sample(n=10000, replace=False, axis=0)
dataset_4 = sample[sample['Rating'] == 4]
dataset_4 = dataset_4.sample(n=10000, replace=False, axis=0)
dataset_5 = sample[sample['Rating'] == 5]
dataset_5 = dataset_5.sample(n=10000, replace=False, axis=0)

In [7]:
dataset=pd.concat([dataset_1, dataset_2, dataset_3, dataset_4, dataset_5])
dataset.describe()

,Rating
count,50000.000000
mean,3.000000
std,1.414228
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [8]:
dataset['Content_review'] = dataset['Content_review'].astype(str)
for i in dataset['Content_review']:
    i = jieba.cut(i, cut_all = True)
dataset['Content_review'] = dataset['Content_review'].apply(lambda x:' '.join(jieba.cut(x)))
dataset

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/c6/wnvc6vnj53lfwp351zd1jcm40000gn/T/jieba.cache
Loading model cost 1.314 seconds.
Prefix dict has been built succesfully.


,Rating,Content_review
244104,1,排队 很久 才 有 位置 ， 餐厅 经理 服务态度 超差 ， 竟然 说 没人 ， 要 客人 ...
184732,1,服务质量 极其 差 ， 从 部长 到 服务员 ， 全部 像 欠 了 500 万 似的 。 4...
453517,1,服务 极差 ！ 全程 被 一个 服务员 搞 到 心情 烦躁 ， 完全 没 理会 味道 ！ 很...
327521,1,晚上 八点 三十 到 店 ， 点菜 要 什么 没什么 ， 服务员 速度 特别 慢 ， 才 很...
461396,1,d 菜 好 咸 ， 吃 完 都 要 饮 超多水 。 服务 又 差 ， 真是 比钱 添 难受 ...
344905,1,做 的 很 正 ， 环境 很 好 ， 服务 也 还 算 nice
297252,1,难食 ！ 难食 ！ ！ 超级 难食 ！ ！ ！ 未试过 甘难食 的 萝卜 酥 ！ ！ 而且 ...
110017,1,一 进门 感觉 还 挺 神秘 的 但是 进去 里面 后 感觉 还好 点 了 几个 菜 速度 ...
170249,1,烂 ， 六个 人 ， 事先 无 讲明 ， 等 坐下 来 了 ， 才 强迫性 地收 八个 人 ...
265611,1,差 ！ ！ ！ ！ 服务态度 差到 不想 再 去 消费 了 ！ ！ ！ ！


In [9]:
#提取特征
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(dataset['Content_review'])
list_tokenized_train = tokenizer.texts_to_sequences(dataset['Content_review'])
x = pad_sequences(list_tokenized_train, maxlen=120)
y = dataset['Rating']

'''MAX_SEQUENCE_LENGTH = 120 # 每条评论最大长度
EMBEDDING_DIM = 200 # 词向量空间维度
VALIDATION_SPLIT = 0.16 # 验证集比例
TEST_SPLIT = 0.2 # 测试集比例

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Content_review'])
sequences = tokenizer.texts_to_sequences(dataset['Content_review'])
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(dataset['Rating']))'''

"MAX_SEQUENCE_LENGTH = 120 # 每条评论最大长度\nEMBEDDING_DIM = 200 # 词向量空间维度\nVALIDATION_SPLIT = 0.16 # 验证集比例\nTEST_SPLIT = 0.2 # 测试集比例\n\nfrom keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing.sequence import pad_sequences\nfrom keras.utils import to_categorical\n\ntokenizer = Tokenizer()\ntokenizer.fit_on_texts(dataset['Content_review'])\nsequences = tokenizer.texts_to_sequences(dataset['Content_review'])\nword_index = tokenizer.word_index\ndata = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)\nlabels = to_categorical(np.asarray(dataset['Rating']))"

In [10]:
'''#划分数据集
from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]'''

'#划分数据集\nfrom sklearn.model_selection import train_test_split\n#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)\n\np1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))\np2 = int(len(data)*(1-TEST_SPLIT))\nx_train = data[:p1]\ny_train = labels[:p1]\nx_val = data[p1:p2]\ny_val = labels[p1:p2]\nx_test = data[p2:]\ny_test = labels[p2:]'

In [11]:
#划分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

from keras.utils import np_utils
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)

In [12]:
'''from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()'''

"from keras.layers import Dense, Input, Flatten, Dropout\nfrom keras.layers import Conv1D, MaxPooling1D, Embedding\nfrom keras.models import Sequential\n\nmodel = Sequential()\nmodel.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))\nmodel.add(Dropout(0.2))\nmodel.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))\nmodel.add(MaxPooling1D(3))\nmodel.add(Flatten())\nmodel.add(Dense(EMBEDDING_DIM, activation='relu'))\nmodel.add(Dense(labels.shape[1], activation='softmax'))\nmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nmodel.summary()"

In [14]:
#模型搭建
from keras.models import Model, Sequential
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten, Bidirectional, GlobalMaxPool1D
embed_size = 128
max_features = 1000
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
#训练模型
model.fit(x_train, y_train, batch_size=500, epochs=5, validation_split=0.2)

Train on 28000 samples, validate on 7000 samples
Epoch 1/5
28000/28000 [==============================] - 121s 4ms/step - loss: -5.1290 - acc: 0.1955 - val_loss: -17.0298 - val_acc: 0.2014
Epoch 2/5
28000/28000 [==============================] - 118s 4ms/step - loss: -28.2793 - acc: 0.2027 - val_loss: -31.7341 - val_acc: 0.2014
Epoch 3/5
28000/28000 [==============================] - 119s 4ms/step - loss: -31.7008 - acc: 0.2027 - val_loss: -31.7345 - val_acc: 0.2014
Epoch 4/5
28000/28000 [==============================] - 126s 5ms/step - loss: -31.7390 - acc: 0.2027 - val_loss: -31.7345 - val_acc: 0.2014
Epoch 5/5
28000/28000 [==============================] - 117s 4ms/step - loss: -31.7603 - acc: 0.2027 - val_loss: -31.7345 - val_acc: 0.2014


In [16]:
accuracy = model.evaluate(x_test, y_test, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))

15000/15000 [==============================] - 17s 1ms/step
test accuracy:0.19433333316197
